한국어 BoW 분류 방법 

In [ ]:
import sys
!{sys.executable} -m pip install konlpy
# 한국어 형태소 분석기 설치

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 4.0/4.0 MB 30.5 MB/s  0:00:00

   ---------------------------------------- 0/3 [lxml]
   ------------- -------------------------- 1/3 [JPype1]
   -------------------------- ------------- 2/3 [konlpy]
   -------------------------- ------------- 2/3 [konlpy]
   ---------------------------------------- 3/3 [konlpy]



In [7]:
# 한국어 형태소 분석기를 이용해서 커스텀 토크나이저 함수 생성
from konlpy.tag import Okt

okt = Okt()

# CountVectorizer에 전달할 한국어 커스텀 토크나이저 함수 정의
def korean_tokenizer(text):
    # okt.morphs()는 텍스트를 형태소 단위로 나누어 리스트로 반환
    return okt.morphs(text)

print(korean_tokenizer("나는 학교에 갑니다"))


['나', '는', '학교', '에', '갑니다']


In [8]:
# CountVectorizer에 커스텀 토크나이저 적용하기
from sklearn.feature_extraction.text import CountVectorizer

# 한국어 예시 데이터
korean_text_train = [
    "오늘 날씨가 정말 좋네요. 영화 보러 갈까요?",
    "이 영화 정말 재미없네요. 비추합니다.",
    "정말 좋은 영화였습니다. 추천합니다."
]

# tokenizer 매개변수에 우리가 만든 함수를 전달!
vect = CountVectorizer(tokenizer=korean_tokenizer)
vect.fit(korean_text_train)

# 어휘 사전 확인
print("한국어 어휘 사전:", vect.vocabulary_)

# BoW 행렬 변환
X_train_korean = vect.transform(korean_text_train)
print("한국어 BoW 행렬:\n", X_train_korean.toarray())

한국어 어휘 사전: {'오늘': 9, '날씨': 4, '가': 2, '정말': 13, '좋네요': 14, '.': 0, '영화': 8, '보러': 5, '갈까': 3, '요': 10, '?': 1, '이': 11, '재미없네요': 12, '비추': 6, '합니다': 17, '좋은': 15, '였습니다': 7, '추천': 16}
한국어 BoW 행렬:
 [[1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 0 0 0]
 [2 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 1]
 [2 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 1]]


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
# 한국어 불용어 처리
# 1. 토크나이저 함수 자체에서 불용어를 제거하는 방법
def korean_tokenizer_with_stopwords(text):
    stop_words = ['는', '가', '이', '을', '를', '은', '.', ',', '?']
    tokens = okt.morphs(text)
    return [token for token in tokens if token not in stop_words and len(token) > 1] # 한 글자 단어도 추가로 제거

# 2. CountVectorizer의 stop_words 매개변수를 사용하는 방법
korean_stop_words = ['는', '가', '이', '을', '를', '은', '.', ',', '?']
vect = CountVectorizer(tokenizer=korean_tokenizer, stop_words=korean_stop_words)